<a href="https://colab.research.google.com/github/zuzka05/stat_learn/blob/main/video3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Let's Build a Quant Trading Strategy - Part 3

In [ ]:
# y_hat = model(x)
# orders = strategy(y_hat)
# execute(orders)

In [ ]:
# Part 1 = Research
# Part 2 = Strategy
# Part 3 = Implementation

## Goals

In [ ]:
# 1. Code it all together in Python (Model + Strategy) (ideally a statically-typed language like rust)
# 2. Show to build a trading system in a scalable way (very easy to create new strategies + streams data)
# 3. Put it live with real money to see how it performs

## Recap

### Part 1: Research

In [2]:
import models
import torch
import research

model = models.LinearModel(3)
#weights_only=True, it ensures it doesn't run any arbitrary code
model.load_state_dict(torch.load('model_weights.pth', weights_only=True))
model.eval()

LinearModel(
  (linear): Linear(in_features=3, out_features=1, bias=True)
)

### AR(3) Model to predict future log return - 12h forecast horizon

In [ ]:
#2 negative coeffs imply a weak MR

In [3]:
research.print_model_params(model)

linear.weight:
[[-0.10395038 -0.06726477  0.02827305]]
linear.bias:
[0.00067121]


### Part 2: Strategy Recap

In [ ]:
# ~14% without any optimization
# 1. Compounding Trade Sizing
# 2. Leverage
# ~14% to >40%
# The increase due to leverage, compound trade sizing and the model edge

### Fundamental Building Block: Tick

In [ ]:
# Abstract Class doesn't have any implementation
# I expect whatever extends this class to implement this method
# How to handle new data coming from the exchange? Order book data coming, open high & close price series
# When there is new info or new data, what should I do?

# It has this polymorphism - rather than encoded to specific types
# Float tick or a string tick or a decimal tick - completely generic - don't have to code specific implementations

In [4]:
from abc import ABC, abstractmethod
from typing import Generic, TypeVar

T = TypeVar('T')  # input type
R = TypeVar('R')  # output type

class Tick(ABC, Generic[T, R]):
    @abstractmethod
    def on_tick(self, val: T) -> R:
        """Handle a new tick and optionally return a result."""
        pass

### Sliding Window: The fundamental data structure

In [ ]:
#It memorizes more recent data points, last 20 data points

#Double-ended queue, allows you to insert at the front and at the back constant time
#Big O notation, it scales constantly irrespective of the size of the data
#Deque Windon -> still generic, input is T and the output is optional of T
#Optional it is either going to be T or not, in RUST it is an option


In [5]:
from collections import deque
from typing import Deque, Optional
import numpy as np

class DequeWindow(Tick[T, Optional[T]], Generic[T]):
    def __init__(self, n: int):
        self._data: Deque[T] = deque(maxlen=n)

    #if it is full then drop data point at point 0
    #append a new data point to it
    #we can append it both to the right and left (front of the queue)
    def on_tick(self, val: T) -> Optional[T]:
        """Append a value and return the oldest value dropped (if any)."""
        dropped = None
        if self.is_full():
            dropped = self._data[0]
        self._data.append(val)
        return dropped

    def is_full(self) -> bool:
        return self._data.maxlen == len(self._data)

    def append_left(self, val: T) -> Optional[T]:
        dropped = None
        if self.is_full():
            dropped = self._data[-1]
        self._data.appendleft(val)
        return dropped

    def to_numpy(self) -> np.ndarray:
        return np.array(self._data)

    def __repr__(self) -> str:
        cls_name = self.__class__.__name__
        return f"{cls_name}(capacity={self._data.maxlen}, values={list(self._data)})"

In [6]:
w = DequeWindow(3)
w

DequeWindow(capacity=3, values=[])

In [7]:
w = DequeWindow(3)
w.on_tick(1)
w

DequeWindow(capacity=3, values=[1])

In [8]:
w = DequeWindow(3)
w.on_tick(1)
w.on_tick(2)
w.on_tick(3)
w

DequeWindow(capacity=3, values=[1, 2, 3])

In [ ]:
#below 1 has been dropped, if we wanted to keep 1 as a local variable we would have installed it here

In [9]:
w = DequeWindow(3)
w.on_tick(1)
w.on_tick(2)
w.on_tick(3)
w.on_tick(4)
w

DequeWindow(capacity=3, values=[2, 3, 4])

In [ ]:
#It memorizes the last 3 values

In [10]:
for i in range(1000000):
    w.on_tick(i)
w

DequeWindow(capacity=3, values=[999997, 999998, 999999])

### Array-based Window

In [ ]:
#It's pretty much the same as Deque but we're shifting the values in the array manually

In [11]:
import numpy as np
from typing import Optional

class NumpyWindow(Tick[T, Optional[T]]):
    def __init__(self, n: int, dtype=np.float64):
        if n <= 0:
            raise ValueError("Capacity must be positive.")
        self._capacity = n
        self._data = np.zeros(n, dtype=dtype)
        self._size = 0

    def on_tick(self, val: float) -> Optional[float]:
        dropped = None

        if self._size < self._capacity:
            self._data[self._size] = val
            self._size += 1
        else:
            dropped = self._data[0]
            # shift left in-place
            for i in range(1, self._capacity):
                self._data[i - 1] = self._data[i]
            self._data[-1] = val

        return dropped


    def __getitem__(self, idx: int) -> float:
        """Index access (0 = oldest)."""
        if not 0 <= idx < self._size:
            raise IndexError("Index out of range.")
        return self._data[idx]

    def __len__(self) -> int:
        return self._size

    def capacity(self) -> int:
        return self._capacity

    def is_full(self) -> bool:
        return self._size == self._capacity

    def values(self) -> np.ndarray:
        return self._data[:self._size]

    def __repr__(self) -> str:
        vals = self.values().tolist()
        return f"{self.__class__.__name__}(capacity={self._capacity}, size={self._size}, values={vals})"


### Benchmark Numpy Window vs Deque Window

In [ ]:
#Do the benchmarking

In [12]:
def benchmark_window(window, n):
    for i in range(n):
        window.on_tick(i)

window_size = 10
n = 5000000

In [ ]:
#numpy is an array so it is a continous memory allocation

In [13]:
%%time
benchmark_window(NumpyWindow(window_size), n)

CPU times: user 7.77 s, sys: 900 µs, total: 7.77 s
Wall time: 7.82 s


In [ ]:
#deque is a linked list which is not-continous
#it's not in line for memory slot

In [14]:
%%time
benchmark_window(DequeWindow(window_size), n)

CPU times: user 717 ms, sys: 0 ns, total: 717 ms
Wall time: 718 ms


### Stream the Last Known Value

In [ ]:
#For each time interval, we want to track the last known trading price
#To do this we need to aggregate the last known value
#The most important is that we're extending tick, to do it we also need to implement on_tick method
#We store the Optional = None, once the tick comes in, we update it

In [15]:
class Last(Tick[T, T], Generic[T]):
    def __init__(self):
        self._value: Optional[T] = None

    def on_tick(self, val: T) -> Optional[T]:
        self._value = val
        return val

    def __repr__(self) -> str:
        cls_name = self.__class__.__name__
        return f"{cls_name}(value={self._value})"

In [16]:
last_val = Last()
last_val

Last(value=None)

In [ ]:
#It's important to stream the last known trading price

In [17]:
last_val = Last()
for i in range(5):
    last_val.on_tick(i)
last_val

Last(value=4)

## Streaming Log Returns

In [ ]:
#Stream log returns, take the last 3 known log returns

### Recap

In [ ]:
#This calculates log returns for price TS
#Adding those up gives, net return positions

In [18]:
ts = [100, 120, 100]
log_returns = [
    np.log(ts[1] / ts[0]),
    np.log(ts[2] / ts[1])
]
log_returns

[np.float64(0.1823215567939546), np.float64(-0.1823215567939546)]

In [19]:
np.sum(log_returns)

np.float64(0.0)

In [21]:
import numpy as np

class LogReturn(Tick[float, Optional[float]], Generic[T]):
    def __init__(self):
        self._window = NumpyWindow(2)

    def on_tick(self, val: float) -> Optional[float]:
        #this is streaming log returns
        #we have 2 price to calculate log returns
        self._window.on_tick(val)
        if self._window.is_full():
            return np.log(self._window[1] / self._window[0])
        else:
            return None

    def __repr__(self) -> str:
        cls_name = self.__class__.__name__
        return f"{cls_name}(window={self._window})"


In [22]:
#Our streaming function is the Log return
f = LogReturn()
f.on_tick(100.0)
f

LogReturn(window=NumpyWindow(capacity=2, size=1, values=[100.0]))

In [23]:
v = f.on_tick(120.0)
v

np.float64(0.1823215567939546)

In [24]:
f

LogReturn(window=NumpyWindow(capacity=2, size=2, values=[100.0, 120.0]))

In [25]:
v = f.on_tick(100.0)
v

np.float64(-0.1823215567939546)

In [ ]:
#It streams intelligently, the memory is always constant size

In [26]:
f

LogReturn(window=NumpyWindow(capacity=2, size=2, values=[120.0, 100.0]))

### Streaming Auto-Regressive Log Returns Lags

In [ ]:
#Lags are previous values in the TS

### Recap

In [ ]:
#The most recent is always on the right

In [27]:
time_series = [0.1, -0.2, -0.3]
lag_1 = time_series[-1]
lag_1

-0.3

In [28]:
lag_2 = time_series[-2]
lag_2

-0.2

In [29]:
lag_3 = time_series[-3]
lag_3

0.1

In [ ]:
#We're extending the tick class as we want to implement on_tick

In [31]:
class LogReturnLags(Tick[float, torch.Tensor]):
    def __init__(self, no_lags: int):
        #the number of lags that we're interested in
        #here we're interested in an arbitrary amount of lags
        #create the DequeWindow and then we stream log returns
        self._lags = DequeWindow(no_lags)
        self._log_return = LogReturn()

    def on_tick(self, val: float) -> torch.Tensor | None:
        log_ret = self._log_return.on_tick(val)
        #if there's log return we append it to the lags
        if log_ret is not None:
            self._lags.append_left(log_ret)
            #regenerate in torch tensor as the model in pytorch not numpy arrays
            #we pass numpy array, with numpy it can do 0-memory copy
            #we allocate the memory as a numpy array and pass it to tensor
            #it doesn't create a copy of it, just does a memory swap, a 0 copy
            #we only do it if the lags is full, otherwise return None
            return torch.tensor(self._lags.to_numpy(), dtype=torch.float32) if self._lags.is_full() else None
        else:
            return None

    def __repr__(self) -> str:
        cls_name = self.__class__.__name__
        return f"{cls_name}(lags={self._lags}, log_return={self._log_return})"

In [ ]:
lags = LogReturnLags(3)
v = lags.on_tick(90)
lags

LogReturnLags(lags=DequeWindow(capacity=3, values=[]), log_return=LogReturn(window=NumpyWindow(capacity=2, size=1, values=[90.0])))

In [ ]:
lags.on_tick(100)
lags

LogReturnLags(lags=DequeWindow(capacity=3, values=[np.float64(0.10536051565782635)]), log_return=LogReturn(window=NumpyWindow(capacity=2, size=2, values=[90.0, 100.0])))

In [ ]:
lags.on_tick(150)
lags

LogReturnLags(lags=DequeWindow(capacity=3, values=[np.float64(0.4054651081081644), np.float64(0.10536051565782635)]), log_return=LogReturn(window=NumpyWindow(capacity=2, size=2, values=[100.0, 150.0])))

In [ ]:
lags.on_tick(110)

tensor([-0.3102,  0.4055,  0.1054])

In [ ]:
[np.log(110/150), np.log(150/100),np.log(100/90)]

[np.float64(-0.3101549283038396),
 np.float64(0.4054651081081644),
 np.float64(0.10536051565782635)]

In [ ]:
lags = LogReturnLags(3)
lags.on_tick(90)
lags.on_tick(100)
lags.on_tick(150)
lags.on_tick(110)
features = lags.on_tick(160)
features

tensor([ 0.3747, -0.3102,  0.4055])

### Streaming features into our model

In [ ]:
X = features
with torch.no_grad():
    y_hat = model(X)
y_hat

tensor([-0.0060])

In [ ]:
y_hat[0]

tensor(-0.0060)

## Build the Trading System

### Using Decimal to represent money

In [ ]:
val = 0.1
total = 0.0
for i in range(10):
    total += val
total

0.9999999999999999

In [ ]:
from decimal import Decimal
dp = Decimal('0.2')
val = Decimal(0.1).quantize(dp)
total = Decimal(0.0).quantize(dp)
for i in range(10):
    total += val
total

Decimal('1.0')

In [ ]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Order:
    sym: str
    signed_qty: Decimal

    def __str__(self) -> str:
        sign = "LONG" if self.signed_qty > 0 else "SHORT"
        return f"Order({sign} {self.signed_qty} {self.sym})"

In [ ]:
from decimal import Decimal

def decimal_sign(d: Decimal) -> int:
    return 1 if d > Decimal(0) else -1

def is_long(x: Decimal) -> bool:
    return decimal_sign(x) > 0

@dataclass(frozen=True)
class Trade:
    sym: str
    signed_qty: Decimal
    price: Decimal
    pnl: Decimal

    def __str__(self) -> str:
        sign = "LONG" if is_long(self.signed_qty) else "SHORT"
        return f"Trade({sign} {self.signed_qty} {self.sym} {self.price} {self.pnl})"

    def is_long(self) -> bool:
        return is_long(self.signed_qty)




In [ ]:
@dataclass
class Position:
    sym: str
    signed_qty: Decimal
    price: Decimal

    def close(self) -> "Order":
        return Order(self.sym, -self.signed_qty)

    def is_long(self) -> bool:
        return is_long(self.signed_qty)

    def unrealized_pnl(self, current_price: Decimal) -> Decimal:
        entry_val = self.price * self.signed_qty
        exit_val = current_price * -self.signed_qty
        return entry_val + exit_val


In [ ]:
from abc import ABC, abstractmethod
from decimal import Decimal

class Account(ABC):
    @abstractmethod
    def balance(self) -> Decimal:
        pass

    @abstractmethod
    def get_position(self, sym: str) -> Optional[Position]:
        pass

In [ ]:
from decimal import Decimal
from typing import Dict, List, Optional

class TestAccount(Account):
    """A simulated account for testing or paper trading."""

    def __init__(self, _balance: Decimal) -> None:
        self._balance = _balance
        self._positions: Dict[str, Position] = {}
        self._trades: List[Trade] = []

    def balance(self) -> Decimal:
        return self._balance

    def get_position(self, sym) -> Optional[Position]:
        return self._positions.get(sym)

    def __repr__(self) -> str:
        return f"TestAccount(balance={self._balance}, positions={self._positions}, trades={self._trades})"


In [ ]:
acc = TestAccount(Decimal(50.0))
acc.balance()

Decimal('50')

In [ ]:
acc

TestAccount(balance=50, positions={}, trades=[])

### Model an Exchange

In [ ]:
from abc import abstractmethod
from decimal import Decimal

class Exchange(Account):
    """Abstract base class representing a trading exchange/broker."""

    @abstractmethod
    def market_order(self, sym: str, signed_qty: Decimal, price: Decimal) -> Trade:
        """Execute a market order and return a Trade result."""
        pass

    @abstractmethod
    def limit_order(self, sym: str, signed_qty: Decimal, price: Decimal, post_only: bool = False) -> Optional[Trade]:
        """Execute a limit order and return a Trade if it crosses book."""
        pass

In [ ]:
from typing import Dict,List

class TestExchange(Exchange):
    _account: TestAccount

    def __init__(self, account: TestAccount):
        self._account = account

    def market_order(self, sym: str, signed_qty: Decimal, price: Decimal) -> "Trade":
        # Update balance and position
        trade = self._update_position(sym, signed_qty, price)
        self._account._balance += trade.pnl
        self._account._trades.append(trade)
        return trade

    def _update_position(self, sym: str, signed_qty, price: Decimal) -> Trade:
        position = self._account._positions.pop(sym, None)
        pnl = Decimal(0.0)
        if position is not None:
            entry_val = position.price * position.signed_qty
            exit_val = price * position.signed_qty
            pnl = exit_val - entry_val
        else:
            self._account._positions[sym] = Position(sym, signed_qty, price)
        return Trade(sym, signed_qty, price, pnl)

    def limit_order(self, sym, signed_qty, price, post_only = False):
        raise Exception("not yet implemented")

    def balance(self) -> Decimal:
        return self._account.balance()

    def get_position(self, sym) -> Optional[Position]:
        return self._account.get_positions(sym)

    def __repr__(self) -> str:
        return f"TestExchange(balance={self.balance()}, positions={self._account._positions}, trades={self._account._trades})"

### Open Position

In [ ]:
exchange = TestExchange(TestAccount(Decimal(50.0)))

price = Decimal(10)
qty = Decimal(5.0)
exchange.market_order('BTCUSDT', qty, Decimal(price))

Trade(sym='BTCUSDT', signed_qty=Decimal('5'), price=Decimal('10'), pnl=Decimal('0'))

In [ ]:
exchange

TestExchange(balance=50, positions={'BTCUSDT': Position(sym='BTCUSDT', signed_qty=Decimal('5'), price=Decimal('10'))}, trades=[Trade(sym='BTCUSDT', signed_qty=Decimal('5'), price=Decimal('10'), pnl=Decimal('0'))])

### Close Position

In [ ]:
price = Decimal(15.0)
exchange.market_order('BTCUSDT', -qty, price)

Trade(sym='BTCUSDT', signed_qty=Decimal('-5'), price=Decimal('15'), pnl=Decimal('25'))

In [ ]:
exchange

TestExchange(balance=75, positions={}, trades=[Trade(sym='BTCUSDT', signed_qty=Decimal('5'), price=Decimal('10'), pnl=Decimal('0')), Trade(sym='BTCUSDT', signed_qty=Decimal('-5'), price=Decimal('15'), pnl=Decimal('25'))])

In [ ]:
entry_notional_value = Decimal(5) * Decimal(10)
entry_notional_value

Decimal('50')

In [ ]:
exit_notional_val = Decimal(5) * Decimal(15)
exit_notional_val

Decimal('75')

In [ ]:
exit_notional_val - entry_notional_value

Decimal('25')

### Build Strategy API

In [ ]:
class Strategy(ABC):
    @abstractmethod
    def on_tick(self, price: float, account: Account) -> Optional[List[Order]]:
        pass

### Implement our strategy

In [ ]:
import torch.nn as nn

class BasicTakerStrat(Strategy):
    def __init__(self,
                 sym: str,
                 model: nn.Module,
                 log_return_lags: LogReturnLags,
                 scale_factor: Decimal = None) -> None:
        self.sym = sym
        self.model = model
        self.log_return_lags = log_return_lags
        if scale_factor is None:
            scale_factor = Decimal(1.0)
        self.scale_factor = Decimal(scale_factor)

    def _signed_compound_trade_size(self, y_hat: float, account: Account, cur_price: Decimal, position: Optional[Position]) -> Decimal:
        dir_signal = np.sign(y_hat)
        cur_balance =  account.balance()
        unrealized_balance = cur_balance + (position.unrealized_pnl(cur_price) if position else Decimal(0.0))
        qty = unrealized_balance / cur_price
        signed_qty = Decimal(dir_signal) * qty
        return signed_qty * self.scale_factor

    def _create_orders(self, y_hat: torch.Tensor, account: Account, price: Decimal) -> List[Order]:
        position = account.get_position(self.sym)
        signed_trade_size = self._signed_compound_trade_size(y_hat.item(), account, price, position)
        open_order = Order(self.sym, signed_trade_size)
        if position is not None:
            close_order = Order(position.sym, -position.signed_qty)
            return [close_order, open_order]
        return [open_order]

    def on_tick(self, price: float, account: Account) -> List[Order]:
        X = self.log_return_lags.on_tick(price)
        if X is not None:
            with torch.no_grad():
                y_hat = self.model(X)
                orders = self._create_orders(y_hat, account, Decimal(price))
                return orders
        return []

In [ ]:
# Window to stream lagged log returns
lags = LogReturnLags(3)
# Create Account
acc = TestAccount(Decimal(100.0))
# Create strategy
strat = BasicTakerStrat('BTCUSDT', model, lags, Decimal(1.0))

# First 12 hour interval - 2025/10/20 00:00
strat.on_tick(10.0, acc)

[]

In [ ]:
# Second 12 hour interval - 2025/10/20 12:00
strat.on_tick(120.0, acc)

[]

In [ ]:
# Third 12 hour interval - 2025/10/21 00:00
strat.on_tick(90.0, acc)

[]

In [ ]:
# Fourth 12 hour interval - 2025/10/21 12:00
orders = strat.on_tick(100, acc)
orders

[Order(sym='BTCUSDT', signed_qty=Decimal('1'))]

### Execute Order

In [ ]:
exchange = TestExchange(acc)
order = orders[0]
exchange.market_order(order.sym, order.signed_qty, 100)

Trade(sym='BTCUSDT', signed_qty=Decimal('1'), price=100, pnl=Decimal('0'))

In [ ]:
exchange

TestExchange(balance=100, positions={'BTCUSDT': Position(sym='BTCUSDT', signed_qty=Decimal('1'), price=100)}, trades=[Trade(sym='BTCUSDT', signed_qty=Decimal('1'), price=100, pnl=Decimal('0'))])

In [ ]:
orders = strat.on_tick(115, acc)
orders

[Order(sym='BTCUSDT', signed_qty=Decimal('-1')),
 Order(sym='BTCUSDT', signed_qty=Decimal('-0.7391304347826086956521739130'))]

In [ ]:
order = orders[0]
exchange.market_order(order.sym, order.signed_qty, 115)

Trade(sym='BTCUSDT', signed_qty=Decimal('-1'), price=115, pnl=Decimal('15'))

In [ ]:
exchange

TestExchange(balance=115, positions={}, trades=[Trade(sym='BTCUSDT', signed_qty=Decimal('1'), price=100, pnl=Decimal('0')), Trade(sym='BTCUSDT', signed_qty=Decimal('-1'), price=115, pnl=Decimal('15'))])

In [ ]:
order = orders[1]
exchange.market_order(order.sym, order.signed_qty, 115)

Trade(sym='BTCUSDT', signed_qty=Decimal('-0.7391304347826086956521739130'), price=115, pnl=Decimal('0'))

In [ ]:
exchange

TestExchange(balance=115, positions={'BTCUSDT': Position(sym='BTCUSDT', signed_qty=Decimal('-0.7391304347826086956521739130'), price=115)}, trades=[Trade(sym='BTCUSDT', signed_qty=Decimal('1'), price=100, pnl=Decimal('0')), Trade(sym='BTCUSDT', signed_qty=Decimal('-1'), price=115, pnl=Decimal('15')), Trade(sym='BTCUSDT', signed_qty=Decimal('-0.7391304347826086956521739130'), price=115, pnl=Decimal('0'))])

In [ ]:
orders = strat.on_tick(100, acc)
orders

[Order(sym='BTCUSDT', signed_qty=Decimal('0.7391304347826086956521739130')),
 Order(sym='BTCUSDT', signed_qty=Decimal('1.039130434782608695652173913'))]

In [ ]:
order = orders[0]
exchange.market_order(order.sym, order.signed_qty, 100)

Trade(sym='BTCUSDT', signed_qty=Decimal('0.7391304347826086956521739130'), price=100, pnl=Decimal('11.08695652173913043478260870'))

In [ ]:
exchange

TestExchange(balance=126.0869565217391304347826087, positions={}, trades=[Trade(sym='BTCUSDT', signed_qty=Decimal('1'), price=100, pnl=Decimal('0')), Trade(sym='BTCUSDT', signed_qty=Decimal('-1'), price=115, pnl=Decimal('15')), Trade(sym='BTCUSDT', signed_qty=Decimal('-0.7391304347826086956521739130'), price=115, pnl=Decimal('0')), Trade(sym='BTCUSDT', signed_qty=Decimal('0.7391304347826086956521739130'), price=100, pnl=Decimal('11.08695652173913043478260870'))])

In [ ]:
order = orders[1]
exchange.market_order(order.sym, order.signed_qty, 100)

Trade(sym='BTCUSDT', signed_qty=Decimal('1.039130434782608695652173913'), price=100, pnl=Decimal('0'))

In [ ]:
exchange

TestExchange(balance=126.0869565217391304347826087, positions={'BTCUSDT': Position(sym='BTCUSDT', signed_qty=Decimal('1.039130434782608695652173913'), price=100)}, trades=[Trade(sym='BTCUSDT', signed_qty=Decimal('1'), price=100, pnl=Decimal('0')), Trade(sym='BTCUSDT', signed_qty=Decimal('-1'), price=115, pnl=Decimal('15')), Trade(sym='BTCUSDT', signed_qty=Decimal('-0.7391304347826086956521739130'), price=115, pnl=Decimal('0')), Trade(sym='BTCUSDT', signed_qty=Decimal('0.7391304347826086956521739130'), price=100, pnl=Decimal('11.08695652173913043478260870')), Trade(sym='BTCUSDT', signed_qty=Decimal('1.039130434782608695652173913'), price=100, pnl=Decimal('0'))])